### Step 1: Create the Ground Truth Graph

In [1]:
import sys
import os
import pandas as pd
# If running in an environment where __file__ is not defined, manually set the path
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(project_dir, 'src'))

import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from causalnex.structure import StructureModel
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

In [2]:
import pandas as pd
data = pd.read_csv('../data/final_dataset.csv')
data.columns

Index(['id', 'order_id', 'driver_id', 'driver_action', 'lat', 'lng',
       'Trip Origin', 'Trip Destination', 'Trip Start Time', 'Trip End Time',
       'trip_duration', 'Trip Origin Lat', 'Trip Origin Lng',
       'Trip Destination Lat', 'Trip Destination Lng', 'weather', 'hour',
       'day_of_week', 'is_weekend', 'Distance_km', 'Trip Duration Hours',
       'Average Speed (km/h)', 'accepted_within_radius',
       'unfulfilled_within_radius', 'origin_cluster', 'destination_cluster'],
      dtype='object')

In [3]:
print(len(data))

41011


In [4]:
print(data['is_weekend'].unique())


[False]


### Thus we don't need is_weekend column

In [5]:
print(data['weather'].unique())


['rain' 'no-rain']


In [6]:
# Select relevant columns
columns = ['id', 'order_id', 'driver_id', 'driver_action', 'lat', 'lng',
       'Trip Origin', 'Trip Destination', 'Trip Start Time', 'Trip End Time',
       'trip_duration', 'Trip Origin Lat', 'Trip Origin Lng',
       'Trip Destination Lat', 'Trip Destination Lng', 'weather', 'hour',
       'day_of_week', 'Distance_km', 'Trip Duration Hours',
       'Average Speed (km/h)', 'accepted_within_radius',
       'unfulfilled_within_radius', 'origin_cluster', 'destination_cluster']

data = data[columns]

#take only the first 1000 rows
data = data.head(1000)
# Convert boolean columns to integers
# data['is_holiday'] = data['is_holiday'].astype(int)
# data['is_weekend'] = data['is_weekend'].astype(int)
# Map 'rain' to 1 and 'no rain' to 0
data['weather'] = data['weather'].map({'rain': 1, 'no-rain': 0})

# Convert the column to int type
data['weather'] = data['weather'].astype(int)

# Rename columns to remove spaces and special characters and this helps to create the StructureModel.
data.rename(columns={
    'Trip Origin Lat': 'Trip_Origin_Lat',
    'Trip Origin Lng': 'Trip_Origin_Lng',
    'Trip Destination Lat': 'Trip_Destination_Lat',
    'Trip Destination Lng': 'Trip_Destination_Lng',
    'Average Speed (km/h)': 'Average_Speed_kmph',
    'Trip Duration Hours': 'Trip_Duration_Hours'
}, inplace=True)


# Split the data into training and hold-out sets
train_data, holdout_data = train_test_split(data, test_size=0.2, random_state=42)

# Save training and holdout sets
train_data.to_csv('train_data.csv', index=False)
holdout_data.to_csv('holdout_data.csv', index=False)


In [7]:
print(len(train_data))

800


In [8]:
print(train_data.head)

<bound method NDFrame.head of       id  order_id  driver_id driver_action       lat       lng  \
29    40    392005     171165      rejected  6.564058  3.370044   
535  546    392033     244161      rejected  6.544610  3.362076   
695  706    392036     243614      rejected  6.433190  3.503240   
557  568    392033     243990      rejected  6.524337  3.360462   
836  847    392041     243581      rejected  6.600685  3.352155   
..   ...       ...        ...           ...       ...       ...   
106  117    392009     245531      rejected  6.661377  3.309885   
270  281    392021     245662      rejected  6.605025  3.294141   
860  871    392041     243476      rejected  6.594846  3.339450   
435  446    392033     244240      rejected  6.517712  3.381011   
102  113    392009     244102      rejected  6.651027  3.300026   

                       Trip Origin             Trip Destination  \
29     6.565087699999999,3.3844415  6.499696300000001,3.3509075   
535            6.5214112,3.3672

In [9]:
import pandas as pd
from causalnex.structure import StructureModel
from causalnex.structure.notears import from_pandas
# from causalnex.structure.notears import from_pandas_dynamic
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

# Load the training data
train_data = pd.read_csv('train_data.csv')

# Define the columns to include in the causal graph
columns = ['trip_duration', 'Trip_Origin_Lat', 'Trip_Origin_Lng',
           'Trip_Destination_Lat', 'Trip_Destination_Lng', 'Distance_km', 'weather',
           'Average_Speed_kmph', 'accepted_within_radius', 'unfulfilled_within_radius']

# Use the selected columns
train_data = train_data[columns]

# Create the ground truth causal graph using the NOTEARS algorithm
ground_truth_sm = from_pandas(train_data, w_threshold=0.80)


In [10]:

# Plot the causal graph
viz = plot_structure(
    ground_truth_sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK
)

viz.show('ground_truth.html')

ground_truth.html


In [11]:
# Function to get parents of a node in a StructureModel
def get_parents(structure_model, node):
    return [edge[0] for edge in structure_model.edges if edge[1] == node]

# Identify direct parents of the target variable in the stable causal graph
target_variable = 'unfulfilled_within_radius'
direct_parents = get_parents(ground_truth_sm, target_variable)
print("Direct parents of the target variable:", direct_parents)


Direct parents of the target variable: ['trip_duration', 'Average_Speed_kmph', 'accepted_within_radius']


### Step2 .Create Graphs with Increasing Fractions of the Data

In [12]:
# Define fractions to use
fractions = [0.1, 0.3, 0.5, 0.7, 0.9]

# Store the structure models
fraction_smodels = []

for frac in fractions:
    # Sample a fraction of the data
    sample_data = train_data.sample(frac=frac, random_state=42)
    
    # Create a causal graph for the sample data
    sm = from_pandas(sample_data, w_threshold=0.80)
    fraction_smodels.append(sm)

    # Optionally plot and save each graph
    viz = plot_structure(
        sm,
        all_node_attributes=NODE_STYLE.WEAK,
        all_edge_attributes=EDGE_STYLE.WEAK
    )
    viz.show(f'fraction_{int(frac*100)}.html')


fraction_10.html
fraction_30.html
fraction_50.html
fraction_70.html
fraction_90.html


### Step 3: Compare Graphs Using Jaccard Similarity Index

In [13]:
def jaccard_similarity(graph1, graph2):
    """Calculate the Jaccard similarity index between two graphs."""
    edges1 = set(graph1.edges)
    edges2 = set(graph2.edges)
    intersection = edges1.intersection(edges2)
    union = edges1.union(edges2)
    return len(intersection) / len(union)

# Calculate Jaccard Similarity Index for each fraction graph
similarity_scores = []

for sm in fraction_smodels:
    score = jaccard_similarity(ground_truth_sm, sm)
    similarity_scores.append(score)

# Print the similarity scores
for frac, score in zip(fractions, similarity_scores):
    print(f'Fraction: {frac*100:.0f}%, Jaccard Similarity Index: {score:.2f}')


Fraction: 10%, Jaccard Similarity Index: 0.25
Fraction: 30%, Jaccard Similarity Index: 0.19
Fraction: 50%, Jaccard Similarity Index: 0.36
Fraction: 70%, Jaccard Similarity Index: 0.43
Fraction: 90%, Jaccard Similarity Index: 0.68


### Interpretation of the Jaccard Similarity Index Scores

* Fraction: 10%, Jaccard Similarity Index: 0.25
    Interpretation: When using only 10% of the data, the resulting causal graph shares 25% of its edges with the ground truth graph. This indicates some similarity but also significant differences, which is expected due to the limited amount of data.

* Fraction: 30%, Jaccard Similarity Index: 0.19
    Interpretation: Using 30% of the data results in a Jaccard Similarity Index of 0.19, which is lower than that of the 10% fraction. This suggests that the causal graph generated from 30% of the data is less similar to the ground truth graph compared to the one generated from 10%. This could be due to random variations in the data or sampling differences.

* Fraction: 50%, Jaccard Similarity Index: 0.36
    Interpretation: With 50% of the data, the similarity increases to 0.36. This indicates that as more data is used, the resulting causal graph becomes more similar to the ground truth graph. However, there is still a significant amount of variation.

* Fraction: 70%, Jaccard Similarity Index: 0.43
    Interpretation: Using 70% of the data yields a Jaccard Similarity Index of 0.43. The similarity continues to improve, indicating that the graph generated from 70% of the data is more similar to the ground truth graph compared to the graphs generated from smaller fractions.

* Fraction: 90%, Jaccard Similarity Index: 0.68
    Interpretation: At 90% of the data, the Jaccard Similarity Index is 0.68. This high similarity score indicates that the causal graph generated from 90% of the data closely resembles the ground truth graph. This suggests that with 90% of the data, most of the significant causal relationships identified in the full dataset are captured.

### Question4

In [33]:
# Function to get parents of a node in a StructureModel
def get_parents(structure_model, node):
    return [edge[0] for edge in structure_model.edges if edge[1] == node]

# Identify direct parents of the target variable in the stable causal graph
target_variable = 'unfulfilled_within_radius'
direct_parents = get_parents(ground_truth_sm, target_variable)
print("Direct parents of the target variable:", direct_parents)


Direct parents of the target variable: ['trip_duration', 'Average_Speed_kmph', 'accepted_within_radius']


In [15]:
import networkx as nx

# Identify the weakly connected components
components = list(nx.weakly_connected_components(ground_truth_sm))
print(f"Number of components: {len(components)}")
print("Components:", components)


Number of components: 1
Components: [{'weather', 'Average_Speed_kmph', 'accepted_within_radius', 'Trip_Destination_Lng', 'Trip_Destination_Lat', 'Distance_km', 'Trip_Origin_Lat', 'trip_duration', 'Trip_Origin_Lng', 'unfulfilled_within_radius'}]


In [16]:
from causalnex.network import BayesianNetwork

# Convert the StructureModel to a BayesianNetwork
bn = BayesianNetwork(ground_truth_sm)


In [17]:
# Fit CPDs to the BayesianNetwork
bn = bn.fit_node_states_and_cpds(train_data, method="BayesianEstimator", bayes_prior="K2")


In [18]:
from causalnex.inference import InferenceEngine

# Create an inference engine for the Bayesian Network
ie = InferenceEngine(bn)

# Perform the intervention for each question

# Perform the intervention (assuming 1km additional distance)
new_distance = train_data['Distance_km'] + 1
# Question 1: Impact of Drivers Moving 1km Every 30 Minutes


In [21]:
states =ie.query()["Distance_km"]

In [ ]:
print(states)

{0.0603554321200701: 0.0, 0.0696208262878737: 0.0, 0.0800506237784369: 0.0, 0.090254529052517: 0.0, 0.1353929437399915: 0.0, 0.1748434791092629: 0.0, 0.1750598193309729: 0.0, 0.231299362311575: 0.0, 0.3318390285569496: 0.0, 0.3981903956773348: 0.0, 0.438444003609278: 0.0, 0.4898543873915751: 0.0, 0.5501764892977402: 0.9999999999999969}


In [ ]:

# Create a new distance array with the 1km increase
increased_distances = train_data['Distance_km'] + 1



In [ ]:

# Assuming 'states' is a dictionary representing discretized distance bins after causal model creation
num_bins = len(states)

intervention_value = {}
for i in range(num_bins):
    current_state = list(states.keys())[i]

    # Intervention strategy: Uniform shift of probability mass to the next bin (adjust as needed)
    if i == num_bins - 1:
        intervention_value[current_state] = 1.0  # Last bin absorbs remaining probability
    else:
        intervention_value[current_state] = 0.0  # Probability in this bin moves to the next
        intervention_value[list(states.keys())[i + 1]] = 1.0  # Probability goes to next bin


In [ ]:

# Perform the intervention (assuming 'ie' is your causal inference engine)
ie.do_intervention("Distance_km", intervention_value)


In [ ]:
states =ie.query()["Distance_km"]
# Create a new distance array with the 1km increase
increased_distances = train_data['Distance_km'] + 1


# Assuming 'states' is a dictionary representing discretized distance bins after causal model creation
num_bins = len(states)

intervention_value = {}
for i in range(num_bins):
    current_state = list(states.keys())[i]

    # Intervention strategy: Uniform shift of probability mass to the next bin (adjust as needed)
    if i == num_bins - 1:
        intervention_value[current_state] = 1.0  # Last bin absorbs remaining probability
    else:
        intervention_value[current_state] = 0.0  # Probability in this bin moves to the next
        intervention_value[list(states.keys())[i + 1]] = 1.0  # Probability goes to next bin

# Perform the intervention (assuming 'ie' is your causal inference engine)
ie.do_intervention("Distance_km", intervention_value)

# Query the effect on the number of unfulfilled requests
result = ie.query()['unfulfilled_within_radius']
print("Effect on unfulfilled requests with 1km movement every 30 minutes:", result)


Effect on unfulfilled requests with 1km movement every 30 minutes: {0: 0.027363184079602115, 1: 0.9726368159203977}


### Summary

* The intervention of moving drivers 1km every 30 minutes results in a very high probability (97.26%) that there will still be unfulfilled requests within the specified radius. * Conversely, there is a low probability (2.74%) that there will be no unfulfilled requests.

* Implications

    - High Probability of Unfulfilled Requests: The high probability of unfulfilled requests remaining after the intervention suggests that simply moving drivers 1km every 30 minutes is not sufficient to significantly reduce the number of unfulfilled requests. Other factors might need to be considered or additional interventions may be necessary to achieve a more substantial impact.

    - Further Analysis Needed: This insight indicates the need for further analysis to identify other potential factors or more effective interventions that could reduce the number of unfulfilled requests more effectively.

2. Knowing Location of Next 20% of Orders within 5km Accuracy

In [ ]:
#  Improved accuracy in order location prediction
states_origin_lat = ie.query()["Trip_Origin_Lat"]
states_origin_lng = ie.query()["Trip_Origin_Lng"]
states_destination_lat = ie.query()["Trip_Destination_Lat"]
states_destination_lng = ie.query()["Trip_Destination_Lng"]

# Example intervention values, improving accuracy
intervention_value_origin_lat = {state: 1/len(states_origin_lat) for state in states_origin_lat}
intervention_value_origin_lng = {state: 1/len(states_origin_lng) for state in states_origin_lng}
intervention_value_destination_lat = {state: 1/len(states_destination_lat) for state in states_destination_lat}
intervention_value_destination_lng = {state: 1/len(states_destination_lng) for state in states_destination_lng}


In [ ]:
# Apply the interventions
ie.do_intervention("Trip_Origin_Lat", intervention_value_origin_lat)
ie.do_intervention("Trip_Origin_Lng", intervention_value_origin_lng)
ie.do_intervention("Trip_Destination_Lat", intervention_value_destination_lat)
ie.do_intervention("Trip_Destination_Lng", intervention_value_destination_lng)


In [ ]:

# Query the effect on the number of unfulfilled requests
result = ie.query()['unfulfilled_within_radius']
print("Effect on unfulfilled requests with improved order location accuracy:", result)


Effect on unfulfilled requests with improved order location accuracy: {0: 0.027363184079602327, 1: 0.9726368159204085}
